In [13]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statements_from_dataframe,extract_data_into_df,create_etl_watermark_table,insert_data_in_batches
from lookups import FileType
from prehook import return_csv_list,create_staging_tables
import prehook

db_session = create_connection()

In [16]:
from enum import Enum

class FileType(Enum):
    CSV = 1

datatype = FileType.CSV
datapath = "C:\\dataproject\\songs_dataset.csv"
df = extract_data_into_df(datatype,datapath)
df
#datapath2 = "C:\\dataproject\\users_dataset.csv"


In [21]:
# import prehook  

# csv_list = prehook.return_csv_list()
# staging_table_name = "songs"  # Replace with your desired staging table name

# # Test the create_staging_tables function
# prehook.create_staging_tables(csv_list, staging_table_name)


In [22]:
# watermark_table_sql_file ="C:\Data_project\SQL_Commands\V2_prehook_Create_ETL_watermark_table.sql"
# data = pd.read_csv("C:\dataproject\songs_dataset.csv")
# batch_size = 1000

# create_etl_watermark_table(db_session, watermark_table_sql_file)

# # Insert data in batches and record the ETL watermark timestamp
# insert_data_in_batches(data, stg_songs_schema_name, stg_songs_table_name, batch_size, 'etl_watermark')

# if db_session:
#     db_session.close()

In [48]:
import pandas as pd
import pandas_handler 
#from pandas_handler import remove_special_characters
# def remove_special_characters(dataframe, column):
#     # Define a regular expression pattern to match special characters
#     regex_pattern = r'[@_!#$%^&*()<>?/\|}{~:]'
#     dataframe[column] = dataframe[column].apply(
#         lambda x: re.sub(regex_pattern, '', str(x)) if pd.notna(x) else x
#     )
#     return dataframe

df = pd.read_csv('C:\dataproject\songs_dataset.csv')

column_to_clean = 'track_name'
column1_to_clean = 'track_artist'
column2_to_clean = 'track_album_name'

df_cleaned = pandas_handler.remove_special_characters(df, column_to_clean)

df_cleaned = pandas_handler.remove_special_characters(df, column1_to_clean)

df_cleaned = pandas_handler.remove_special_characters(df, column2_to_clean)

df = df_cleaned


In [62]:
special_character = '¼'

# Create a Boolean Series indicating whether the special character is present in each row
is_special_character_present = df['track_artist'].str.contains(special_character, na=False)

# Filter the DataFrame to get rows containing the special character
rows_with_special_character = df[is_special_character_present]

# Print or use rows_with_special_character as needed
rows_with_special_character

df.to_csv('N.csv', index=False)

In [72]:
def sort_dataframe_by_release_date(df, date_column_name):
    df[date_column_name] = pd.to_datetime(df[date_column_name], format='%m/%d/%Y')
    df.sort_values(by=[date_column_name], ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# Sort your existing DataFrame by the release date column
sorted_df = sort_dataframe_by_release_date(df, 'track_album_release_date')

# Print the sorted DataFrame
print(sorted_df)

ValueError: time data "2012" doesn't match format "%m/%d/%Y", at position 96. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# track_name_to_remove = 'Wish You Well with Sigala'

# # Check if the track name to remove exists in the DataFrame
# if track_name_to_remove in df['track_name'].values:
#     # Find the index of the row to remove
#     index_to_remove = df[df['track_name'] == track_name_to_remove].index[0]
#     # Remove the row using the .drop() method
#     df = df.drop(index_to_remove)
# else:
#     print(f"Record with track name '{track_name_to_remove}' not found in the DataFrame.")

# df.to_csv('Na.csv', index=False)
